# Project 10

* 영화 리뷰를 분석하여 영화의 재미 요소를 분석. (영화 장르 분석 아님)
* 전체 영화 리뷰 학습.
* 영화를 구성하는 요소를 가리키는 기저 단어를 찾고, 각 재미 요소를 결정짓는 성분 분석.
* 위 분석을 통해 특정 영화의 리뷰에 등장하는 단어를 학습시켜 재미 요소를 찾을 수 있는지 확인 

In [1]:
import gensim 
from os import walk

file_list = []
current_path = "../group_20/aclImdb/train/"

def read_input(current_path, file_list):
    for (dirpath, dirnames, _) in walk(current_path):
        for dirname in dirnames:
            for (_, _, filenames) in walk(current_path + dirname):
                for filename in filenames:
                    with open(current_path + dirname + '/' + filename, encoding='utf-8') as file:
                        for j, line in enumerate(file):
                            yield gensim.utils.simple_preprocess(line)

document = list(read_input(current_path, file_list))

Imdb_review_model = gensim.models.Word2Vec (document, size=50, window=10, min_count=2, workers=4)
Imdb_review_model.train(document, total_examples=len(document), epochs=10)

C:\Users\hyosa\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


(129368479, 169992700)

In [2]:
Imdb_review_model.wv['fun']

array([ 1.9626734 ,  4.2399545 , -2.8024628 ,  4.130389  ,  2.0931091 ,
        2.7943933 , -2.916158  ,  4.703028  ,  5.566028  , -0.43444416,
        2.4839494 ,  0.9874864 ,  0.70386666, -5.877556  , -4.0919476 ,
        1.8312435 , -0.38619152, -0.61231303,  2.0354528 ,  0.66812974,
       -1.26193   ,  0.09878013,  4.0229554 ,  0.04197988, -1.4265617 ,
       -3.799894  ,  2.2126617 ,  2.9075525 , -0.28324008, -3.6375926 ,
        5.0145974 , -0.88085085,  1.6820583 ,  1.602927  ,  1.7387145 ,
       -0.4052896 , -2.5065067 ,  6.40973   ,  0.5587249 , -3.083036  ,
        1.6676232 , -2.4113271 ,  1.9121791 ,  4.443365  ,  3.2512238 ,
       -4.8519506 , -0.39640316, -0.57349974, -3.0464766 , -1.1380874 ],
      dtype=float32)

In [4]:
Imdb_review_model.wv.most_similar(positive = ['fun'], topn=20)

[('entertaining', 0.7197267413139343),
 ('funny', 0.7046216726303101),
 ('enjoyable', 0.7038799524307251),
 ('laughs', 0.6253747940063477),
 ('watchable', 0.6122944951057434),
 ('scary', 0.6107514500617981),
 ('campy', 0.6027714610099792),
 ('good', 0.5888934135437012),
 ('worth', 0.5884804725646973),
 ('laugh', 0.5869355201721191),
 ('amusing', 0.586571455001831),
 ('cheesy', 0.5811032652854919),
 ('popcorn', 0.573393702507019),
 ('addictive', 0.5638667345046997),
 ('silliness', 0.5594998598098755),
 ('lighthearted', 0.5562014579772949),
 ('comedy', 0.5541406869888306),
 ('worthwhile', 0.5514662265777588),
 ('humor', 0.5501068234443665),
 ('goofy', 0.5451074242591858)]

In [12]:
Imdb_review_model.wv.most_similar(negative = ['good'], topn=20)

[('mullers', 0.6327378749847412),
 ('dispensation', 0.6283209323883057),
 ('neurasthenia', 0.604059100151062),
 ('interrogates', 0.6011544466018677),
 ('epithet', 0.600537896156311),
 ('segregating', 0.5952950716018677),
 ('midori', 0.5848098993301392),
 ('pronouncement', 0.5810747742652893),
 ('designation', 0.5804493427276611),
 ('paralysis', 0.5787739753723145),
 ('notify', 0.5767640471458435),
 ('accosts', 0.570834755897522),
 ('responders', 0.570712685585022),
 ('involvements', 0.5705764293670654),
 ('kahlil', 0.5671861171722412),
 ('christmanish', 0.5615667104721069),
 ('freccia', 0.5600200295448303),
 ('herati', 0.5593158006668091),
 ('hicksville', 0.5579496026039124),
 ('avalyn', 0.5576185584068298)]

미리 학습 시킨 모델에서 리뷰에 등장하는 모든 단어와 기저 단어 (예를 들어 entertaining, horror, cute, love, action) 간의 유사성 (코사인 값)의 평균을 구해서 영화의 재미 요소를 분석

In [66]:
import numpy as np

wordcat = ['entertaining', 'horror', 'cute', 'love', 'action']
v = []
for word in wordcat:
    v.append(Imdb_review_model.wv[word])
    
vocablist = []
for word, vect in Imdb_review_model.wv.vocab.items():
    vocablist.append(word)

file_list = []
current_path = "./"
sumdot = [0,0,0,0,0]
numsum = [0,0,0,0,0]
for (_, _, filenames) in walk(current_path):
    with open("1.txt", encoding='utf-8') as file:
        for j, line in enumerate(file):
            wordlist = gensim.utils.simple_preprocess(line)
            for word in wordlist:
                if word in vocablist:
                    for i in range(len(v)):
                        dotnum = np.dot(v[i], Imdb_review_model.wv[word]) / np.linalg.norm(v[i]) / np.linalg.norm(Imdb_review_model.wv[word])
                        #print(dotnum)
                        sumdot[i] += dotnum
                        #print(sumdot[i])
                        numsum[i] += 1
                        #print(numsum[i])
print(np.array(sumdot)/np.array(numsum))

[0.14256373 0.10643349 0.08020781 0.04523052 0.11495   ]
